In [1]:
import nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from transformers import BertTokenizerFast, BertModel, AdamW
from torch.utils.data import TensorDataset, DataLoader

import time
from tqdm.notebook import tqdm

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

eng_dict = nltk.corpus.words.words('en')


cuda:0


In [2]:


tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

bert_model = BertModel.from_pretrained('bert-base-uncased')
num_layers = len(bert_model.encoder.layer)  # Adjust based on your BERT model
for i in range(0, num_layers):
    if i >= num_layers - 0:
        for param in bert_model.encoder.layer[i].parameters():
            param.requires_grad = True
    else:
        for param in bert_model.encoder.layer[i].parameters():
            param.requires_grad = False


# initialguess for the archetecture is [823, 512, 256, 128, 1]
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(MLP, self).__init__()
        self.fc1  = nn.Linear(input_dim, hidden_dim1)
        self.fc2  = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3  = nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4  = nn.Linear(hidden_dim3, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim1)
        self.batchnorm2 = nn.BatchNorm1d(hidden_dim2)
        self.batchnorm3 = nn.BatchNorm1d(hidden_dim3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x
            
        
class CombinedModel(nn.Module):
    def __init__(self, bert_model, mlp):
        super(CombinedModel, self).__init__()
        self.bert = bert_model
        self.mlp = mlp

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        logits = self.mlp(last_hidden_state)
        return logits
            
        
def train(model, train_loader, optimizer, loss_fn):
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    for batch in tqdm(train_loader):
        input_ids, attention_mask, labels = [t.to(device) for t in batch]
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask) #, labels=labels)
        loss = loss_fn(outputs.to(device), labels.type(torch.LongTensor).to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss
        train_correct += (torch.max(outputs.data, dim=1)[1] == labels).sum().item()
        train_total += len(labels)
        
    train_loss = train_loss / train_total
    train_accuracy = 100 * train_correct / train_total
    return train_loss, train_accuracy


def validation(model, val_loader, loss_fn):
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    y_pred = []
    y_true = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids, attention_mask, labels = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask) #, labels=labels)
            loss = loss_fn(outputs.to(device), labels.type(torch.LongTensor).to(device))
            val_loss += loss
            val_correct += (torch.max(outputs.data, dim=1)[1] == labels).sum().item()
            val_total += len(labels)
            
            y_pred.extend(torch.softmax(outputs.data, dim=1)[:, 1].cpu().numpy())  # Assuming class 1 probability
            y_true.extend(labels.cpu().numpy())

            val_correct += (torch.max(outputs.data, dim=1)[1] == labels).sum().item()
            val_total += len(labels)
    
    y_pred_label = [int(num >= 0.5) for num in y_pred]
    roc_auc = roc_auc_score(y_true, y_pred_label)
    precision = precision_score(y_true, y_pred_label)
    recall = recall_score(y_true, y_pred_label)
    f1 = f1_score(y_true, y_pred_label)
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    val_loss = val_loss / val_total
    val_accuracy = 100 * val_correct / val_total
    return y_pred_label, val_loss, val_accuracy, roc_auc, precision, recall, f1, fpr, tpr


def tokenize_and_encode(tokenizer, comments, labels, max_length=128): 
    input_ids = [] 
    attention_masks = [] 

    for comment in comments: 
        encoded_dict = tokenizer.encode_plus( 
            comment,  
            add_special_tokens=True, 
            max_length=max_length, 
            pad_to_max_length=True, 
            return_attention_mask=True, 
            return_tensors='pt'
        ) 

        input_ids.append(encoded_dict['input_ids']) 
        attention_masks.append(encoded_dict['attention_mask']) 

    input_ids = torch.cat(input_ids, dim=0) 
    attention_masks = torch.cat(attention_masks, dim=0) 

    labels = torch.tensor(labels, dtype=torch.float32) 
    
    return input_ids, attention_masks, labels 


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:

mlp = MLP(768, 512, 256, 128, 2)
model = CombinedModel(bert_model, mlp).to(device)
checkpoint = torch.load('/kaggle/input/llm-detect/checkpoint.pth')
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

df = pd.read_csv('/kaggle/input/human-vs-llm-text-corpus/data.csv', usecols=['text', 'source']) 
df['source'] = df['source'].apply(lambda x: 0 if x == 'Human' else 1)
df = df.rename(columns={'source': 'generated'})

max_len = 400
test_input_ids, test_attention_masks, test_labels = tokenize_and_encode( 
    tokenizer, 
    df.text.tolist(), 
    df.generated.tolist(),
    max_length = max_len
) 

test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels) 
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

predictions, loss, acc, roc_auc, precision, recall, f1, fpr, tpr = validation(model, test_loader, loss_fn)

print(f"Loss: {loss:.4f}, Acc: {acc:.4f}")
print(f"AOC ROC: {roc_auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, ")

df = pd.read_csv('/kaggle/input/human-vs-llm-text-corpus/data.csv')
df['predictions'] = predictions
df.to_csv('output.csv', index=False)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/24654 [00:00<?, ?it/s]

Loss: 0.0335, Acc: 67.2780
AOC ROC: 0.6406, Precision: 0.6472, Recall: 0.9122, F1: 0.7572, 
[1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()